In [2]:
import time 
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import math
import pandas as pd
import warnings
import pickle
import GetSalary
import importlib
import scrape2_2 as scr2
import driver_config as dc
import numpy as np
import scipy.interpolate as si
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
importlib.reload(scr2)
importlib.reload(dc)

# Remove DevTools warning
#chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])


<module 'driver_config' from 'c:\\Users\\ckelaid\\Documents\\Scraping\\driver_config.py'>

In [3]:
# Helper functions

def human_like_mouse(action:ActionChains, startElem):
    # B-spline function
    # Curve base:
    points = [[0, 0], [0, 2], [2, 3], [4, 0], [6, 3], [8, 2], [8, 0]]
    points = np.array(points)

    x = points[:,0]
    y = points[:,1]


    t = range(len(points))
    ipl_t = np.linspace(0.0, len(points) - 1, 100)

    x_tup = si.splrep(t, x, k=3)
    y_tup = si.splrep(t, y, k=3)

    x_list = list(x_tup)
    xl = x.tolist()
    x_list[1] = xl + [0.0, 0.0, 0.0, 0.0]

    y_list = list(y_tup)
    yl = y.tolist()
    y_list[1] = yl + [0.0, 0.0, 0.0, 0.0]

    x_i = si.splev(ipl_t, x_list) # x interpolate values
    y_i = si.splev(ipl_t, y_list) # y interpolate values

    
    start_element = startElem

    action.move_to_element(start_element)
    action.perform()

    c = 5
    i = 0
    for mouse_x, mouse_y in zip(x_i, y_i):
        action.move_by_offset(mouse_x,mouse_y)
        action.perform()
        #print("Move mouse to, %s ,%s" % (mouse_x, mouse_y))   
        i += 1    
        if i == c:
            break




def bypassCaptcha1():

    # Switch to captcha iframe
    iframe = browser.find_elements(By.TAG_NAME,'iframe')[0]
    browser.switch_to.frame(iframe)
    iframe = browser.find_elements(By.TAG_NAME,'iframe')[0]
    browser.switch_to.frame(iframe)
    
    # Captcha check button
    captcha_check = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='recaptcha-anchor']")))
    browser.implicitly_wait(12)
    action =  ActionChains(browser)
    # Simulate human like mouse movements
    human_like_mouse(action, captcha_check)
    captcha_check.click() # Click captcha check


    browser.implicitly_wait(8)
    action =  ActionChains(browser)
    human_like_mouse(action, captcha_check)

    browser.switch_to.default_content()

    print('\nSuccessfuly bypassed Captcha 1!\n')




# Check if we get second captcha check
#  if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):

def bypassCaptcha2():

    t = random.randint(8,10)
    browser.implicitly_wait(t)

    iframe2 = browser.find_elements(By.TAG_NAME,"iframe")[0]
    browser.switch_to.frame(iframe2)
    iframe3 = browser.find_elements(By.XPATH,"//iframe[@title='recaptcha challenge expires in two minutes']")[0]
    browser.switch_to.frame(iframe3)
    # Find nearest element to #shadow-root (closed)
    elem = browser.find_element(By.XPATH, '//div[@class="button-holder help-button-holder"]')
    browser.implicitly_wait(10)
    elem.click() # click on it
    browser.switch_to.default_content()

    print('\nSuccessfuly bypassed Captcha 2!\n')  

## Scraping by keyword

In [4]:
category = 'workday'

base = 'https://www.higheredjobs.com/search/advanced_action.cfm?Remote=&Keyword='
page_num = 1
page = '&PosType=&InstType=&JobCat=&Region=0&SubRegions=&Metros=&OnlyTitle=0&JobCatType=&StartRow='+str(page_num)+'&SortBy=1&NumJobs=100&CatType='

url = base+category+page

## Scrape page links

In [5]:
print(category)

workday


In [14]:
# Scrape page links

browser = dc.initialize_driver()
t = random.randint(8,10)
WebDriverWait(browser, t)
browser.get(url)
try:
    # wait for page to load
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'text-nowrap'))
    )
    max_jobs = browser.find_element(By.CLASS_NAME, 'text-nowrap').text.split(' ')[5]
    max_jobs = int("".join(max_jobs.split(','))) # remove ',' so that we can convert to int 
    num_pages = math.ceil(max_jobs/100) # /100 because we iterate 100 links per page
    browser.quit()

    page_links = []
    page_links.append(url) #1st page

    # for page 2 until the last page
    for p in range(1, num_pages): #=> we want to +100 for as many pages as we iterate through to the page link, starting at the 2nd page
        page_num = page_num + 100
        page = '&PosType=&InstType=&JobCat=&Region=0&SubRegions=&Metros=&OnlyTitle=0&JobCatType=&StartRow='+str(page_num)+'&SortBy=1&NumJobs=100&CatType='
        url = base+category+page
        page_links.append(url)

    print("There are {} pages".format(len(page_links)))
    # Save page links
    with open('Page_Links_'+category+'.pkl', 'wb') as f:
                    pickle.dump(page_links, f)

except Exception: 
    # Check that exception is b/c of Captcha
    if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):
        ppp = 0
        while ppp < 1:
            try:
                WebDriverWait(browser, 60)
                iframe = browser.find_elements(By.TAG_NAME,'iframe')[0]
                WebDriverWait(browser, 10)
                browser.switch_to.frame(iframe)
                WebDriverWait(browser, 10)
                browser.find_element(By.XPATH, "//*[@id='recaptcha-anchor']").click()
                time.sleep(20) # manually solve image challenge => that worked
                ppp = 1
            except Exception:
                ppp = 0
        max_jobs = browser.find_element(By.CLASS_NAME, 'text-nowrap').text.split(' ')[5]
        max_jobs = int("".join(max_jobs.split(','))) # remove ',' so that we can convert to int 
        num_pages = math.ceil(max_jobs/100) # /100 because we iterate 100 links per page
        browser.quit()

        page_links = []
        page_links.append(url) #1st page

        # for page 2 until the last page
        for p in range(1, num_pages): #=> we want to +100 for as many pages as we iterate through to the page link, starting at the 2nd page
            page_num = page_num + 100
            page = '&PosType=&InstType=&JobCat=&Region=0&SubRegions=&Metros=&OnlyTitle=0&JobCatType=&StartRow='+str(page_num)+'&SortBy=1&NumJobs=100&CatType='
            url = base+x+page
            page_links.append(url)

        print("There are {} pages".format(len(page_links)))
        # Save page links
        with open('Page_Links_'+category+'.pkl', 'wb') as f:
                    pickle.dump(page_links, f)
        


There are 54 pages


In [12]:
#job_links = []
with open('Job_links_'+category+'.pkl', 'rb') as f:
    job_links = pickle.load(f)

with open('Page_Links_'+category+'.pkl', 'rb') as f:
    page_links = pickle.load(f)

job_attrs = []
not_scraped = 0
scraped = 0
smile = 0

len(job_links), len(page_links) # => page_links[24:], we are on page 23, but let's skip it and go on the next to avoid duplicates, we miss 67 jobs

(2323, 54)

In [13]:
# Scraping job links + cleaning job links + scraping job attributes

for link in page_links[24:]:
    t = random.randint(8,10)
    time.sleep(t)
    browser = dc.initialize_driver()
    browser.get(link)
    try:
        # wait for page to load
        WebDriverWait(browser, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'col-sm-7'))
        )
        # get job links
        div = browser.find_elements(By.CLASS_NAME, 'col-sm-7')
        for d in div:
            job_links.append(d.find_element(By.CSS_SELECTOR, 'a').get_attribute('href'))
        browser.quit()
        with open('Job_links_'+category+'.pkl', 'wb') as f:
            pickle.dump(job_links, f)
    except Exception:
        # print('bypassing Captcha')
        # WebDriverWait(browser, 10).until(
        #     EC.presence_of_all_elements_located((By.TAG_NAME,'iframe'))
        # )
        # Check that exception is b/c of Captcha
        if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):

            #bypassFullCaptcha()
            bypassCaptcha1()
            
            # Check if we get second captcha check
            if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):
                
                bypassCaptcha2()
                time.sleep(10)

        try:
            print('Looking for job link\n')
                
            # Successfully moved out of Captcha

            # wait for page to load
            WebDriverWait(browser, 20).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'col-sm-7'))
            ) 
            # get job links
            div = browser.find_elements(By.CLASS_NAME, 'col-sm-7')
            for d in div:
                job_links.append(d.find_element(By.CSS_SELECTOR, 'a').get_attribute('href'))
            browser.quit()
            with open('Job_links_'+category+'.pkl', 'wb') as f:
                pickle.dump(job_links, f)
        
        except Exception:
            # API limit exceeded and need to wait, pause 20 minutes and try again
            browser.quit()
            print('Pausing 20 minutes, buster API overused!')
            time.sleep(1200) # wait 20 minutes :( looooooong

            pp = 0
            aa = 0
            while pp < 1:
                try:
                    browser = dc.initialize_driver()
                    browser.get(link)

                    #browser.switch_to.default_content()
                    
                    if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):
                        print('Trying again')

                        time.sleep(10)

                        bypassCaptcha1()

                        # Check if we get second captcha check
                        if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):

                            bypassCaptcha2()
                            time.sleep(10)

                        print('Looking for job link\n')
                            
                        # Successfully moved out of Captcha

                        # wait for page to load
                        WebDriverWait(browser, 20).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'col-sm-7'))
                        ) 
                        # get job links
                        div = browser.find_elements(By.CLASS_NAME, 'col-sm-7')
                        for d in div:
                            job_links.append(d.find_element(By.CSS_SELECTOR, 'a').get_attribute('href'))
                        browser.quit()
                        with open('Job_links_'+category+'.pkl', 'wb') as f:
                            pickle.dump(job_links, f)
                        pp = 1
                    
                except Exception:
                    time.sleep(60)
                    browser.implicitly_wait(7)
                    browser.quit()
                    pp = 0
                    aa = aa+1
                    if aa == 5:
                        print('Pausing for 30')
                        time.sleep(1800) # 30 minutes

                    if aa == 6:
                        print('Pausing for 3 hours!')
                        time.sleep(10800)
                    
                    if aa > 6:
                        print('Pause for 2 min intervals until it works')
                        time.sleep(120)
                    
                    if aa == 12:
                        print('Pause for 3 hours')
                        time.sleep(10800)

# Done with job links
print("Job_links: ",len(job_links))

######################################

# Cleaning job_links

######################################

# Deleting wrong entries
del_ix = []
for i in range(len(job_links)):
    if job_links[i] == 'javascript:;':
        del_ix.append(i) 

# Also check that no job_links == category_page url
for j in range(len(job_links)):
    if job_links[j] == 'https://www.higheredjobs.com/search/advanced_action.cfm?Remote=&Keyword='+category+'&PosType=&InstType=&JobCat=&Region=0&SubRegions=&Metros=&OnlyTitle=0&JobCatType=&SortBy=1&StartRow=1':
        del_ix.append(j)

if len(del_ix) > 0:
    # delete 'javascript:;' entries & incorrect url entries
    del job_links[del_ix[0]]

    drop = 1
    for i in range(1, len(del_ix)):
        del job_links[del_ix[i]-drop]
        drop = drop + 1

print("There are {} jobs to scrape".format(len(job_links)))

# Save job links
with open('Job_links_'+category+'.pkl', 'wb') as f:
            pickle.dump(job_links, f)


# Done cleaning job links

print("Job_links: ",len(job_links))

######################################


####### Scraping job attributes ######


######################################

# job_attrs = []
# not_scraped = 0
# scraped = 0
# smile = 0


for link in job_links: #start at job 212, bc job 1 is index 0
    t = random.randint(8,10)
    time.sleep(t)
    browser = dc.initialize_driver()
    browser.get(link)

    try:
        # wait for page to load
        WebDriverWait(browser, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'row'))
        )
        del_div = browser.find_elements(By.CLASS_NAME, 'row')
        # Check if post has been deleted
        if del_div[6].text != 'We require users to verify the reCaptcha below to view deleted positions.\nRelated Searches:\nBusiness and Financial Services\nCreate your free job search account\nReceive new jobs by email\nPost your resume/CV\nTrack your applications\nJoin Now\nHave an account? Sign in':

            job_title = browser.find_element(By.ID, 'jobtitle-header').text
            job_loc = browser.find_element(By.CLASS_NAME, 'job-loc').text
            div = browser.find_elements(By.ID, 'jobAttrib')
            job_attr = div[0].find_element(By.CLASS_NAME, 'job-info').text.split('\n') # get job data
       
            div = browser.find_elements(By.ID, 'job')
            job_desc = div[0].find_element(By.ID, 'jobDesc').text#.split('\n')
            
            job_attr = [job_title] + [job_loc] + job_attr # puts title at the begining of the list
            job_attr.append(job_desc) # Append job_description
            job_attrs.append(job_attr) # Append job details as list inside another list 
            
            scraped = scraped + 1

            browser.quit()
            smile = smile +1
            if smile == 1:
                print('\nScraping away...\n')
                print('0  0\n')
                print('\__/\n')
            if smile % 100 == 0:
                print("{} jobs have been scraped so far...\n".format(scraped))
                print('0  0\n')
                print('\__/\n')
            
            # Save progress (every job info)
            #if smile % 500 == 0: # Every 500 jobs save as pickle
                # Store data (serialize)
            with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                pickle.dump(job_attrs, f)

        elif IndexError:
            print('\nJob {} has been deleted'.format(smile))
            not_scraped = not_scraped + 1

            browser.quit()
            smile = smile +1
            if smile == 1:
                print('\nScraping away...\n')
                print('0  0\n')
                print('\__/\n')
            if smile % 100 == 0:
                print("{} jobs have been scraped so far...\n".format(scraped))
                print('0  0\n')
                print('\__/\n')
            
            # Save progress (every job info)
            #if smile % 500 == 0: # Every 500 jobs save as pickle
                # Store data (serialize)
            with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                pickle.dump(job_attrs, f)

    except Exception: # If cannot find element

        # # wait for page to load
        # WebDriverWait(browser, 20).until(
        #     EC.presence_of_element_located((By.CLASS_NAME, 'iframe'))
        # ) 
               
        # See if exception is related to captcha
        if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):
            
            #bypassFullCaptcha()
            bypassCaptcha1()
            
            # Check if we get second captcha check
            if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):
                
                bypassCaptcha2()
                time.sleep(10)
        
        try:
            print('Looking for job info {}\n'.format(smile))
            print('\nNot scraped is', not_scraped)
            # Successfully moved out of Captcha

            # wait for page to load
            WebDriverWait(browser, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'row'))
            )
            del_div = browser.find_elements(By.CLASS_NAME, 'row')
            # Check if post has been deleted
            if del_div[6].text != 'We require users to verify the reCaptcha below to view deleted positions.\nRelated Searches:\nBusiness and Financial Services\nCreate your free job search account\nReceive new jobs by email\nPost your resume/CV\nTrack your applications\nJoin Now\nHave an account? Sign in':

                job_title = browser.find_element(By.ID, 'jobtitle-header').text
                job_loc = browser.find_element(By.CLASS_NAME, 'job-loc').text
                div = browser.find_elements(By.ID, 'jobAttrib')
                job_attr = div[0].find_element(By.CLASS_NAME, 'job-info').text.split('\n') # get job data
                        #job_attr = [job_title] + job_attr
                div = browser.find_elements(By.ID, 'job')
                job_desc = div[0].find_element(By.ID, 'jobDesc').text#.split('\n')
                    #job_attr.append(job_desc)
                    #job_attrs.append(job_attr)
                
            #else: # if post is delete
                job_attr = [job_title] + [job_loc] + job_attr # puts title at the begining of the list
                job_attr.append(job_desc) # Append job_description
                job_attrs.append(job_attr) # Append job details as list inside another list 
                
                scraped = scraped + 1
                
                browser.quit()
                smile = smile +1
                if smile == 1:
                    print('\nScraping away...\n')
                    print('0  0\n')
                    print('\__/\n')
                if smile % 100 == 0:
                    print("{} jobs have been scraped so far...\n".format(scraped))
                    print('0  0\n')
                    print('\__/\n')
                
                # Save progress (every job info)
                #if smile % 500 == 0: # Every 500 jobs save as pickle
                    # Store data (serialize)
                with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                    pickle.dump(job_attrs, f)

            elif IndexError:
                print('\nJob {} has been deleted'.format(smile))
                not_scraped = not_scraped + 1

                browser.quit()
                smile = smile +1
                if smile == 1:
                    print('\nScraping away...\n')
                    print('0  0\n')
                    print('\__/\n')
                if smile % 100 == 0:
                    print("{} jobs have been scraped so far...\n".format(scraped))
                    print('0  0\n')
                    print('\__/\n')
                
                # Save progress (every job info)
                #if smile % 500 == 0: # Every 500 jobs save as pickle
                    # Store data (serialize)
                with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                    pickle.dump(job_attrs, f)
            
        except Exception:

            try:
                if del_div[6].text.startswith('We require users to verify the reCaptcha below to view deleted positions'):
                    
                    print('\nJob {} has been deleted'.format(smile))
                    not_scraped = not_scraped + 1

                    browser.quit()
                    smile = smile +1
                    if smile == 1:
                        print('\nScraping away...\n')
                        print('0  0\n')
                        print('\__/\n')
                    if smile % 100 == 0:
                        print("{} jobs have been scraped so far...\n".format(scraped))
                        print('0  0\n')
                        print('\__/\n')
                    
                    # Save progress (every job info)
                    #if smile % 500 == 0: # Every 500 jobs save as pickle
                        # Store data (serialize)
                    with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                        pickle.dump(job_attrs, f)
            
            except Exception:
            # If exception is not b/c job is deleted then we need to pause
                # API limit exceeded and need to wait, pause 20 minutes and try again
                browser.quit()
                print('Pausing 20 minutes, buster API overused!')
                time.sleep(1200) # wait 20 minutes :( looooooong

                pp = 0
                aa = 0
                while pp < 1:
                    try:
                        browser = dc.initialize_driver()
                        browser.get(link)

                        #browser.switch_to.default_content()
                        
                        if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):
                            print('Trying again on job ', smile)

                            time.sleep(10)

                            bypassCaptcha1()

                            bypassCaptcha2()
                            time.sleep(10)

                            print('Looking for job info {}\n'.format(smile))
                            print('\nNot scraped is', not_scraped)
                                
                            # Successfully moved out of Captcha

                            # wait for page to load
                            WebDriverWait(browser, 20).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'row'))
                            )
                            del_div = browser.find_elements(By.CLASS_NAME, 'row')
                            # Check if post has been deleted
                            if del_div[6].text != 'We require users to verify the reCaptcha below to view deleted positions.\nRelated Searches:\nBusiness and Financial Services\nCreate your free job search account\nReceive new jobs by email\nPost your resume/CV\nTrack your applications\nJoin Now\nHave an account? Sign in':

                                job_title = browser.find_element(By.ID, 'jobtitle-header').text
                                job_loc = browser.find_element(By.CLASS_NAME, 'job-loc').text
                                div = browser.find_elements(By.ID, 'jobAttrib')
                                job_attr = div[0].find_element(By.CLASS_NAME, 'job-info').text.split('\n') # get job data
                                        #job_attr = [job_title] + job_attr
                                div = browser.find_elements(By.ID, 'job')
                                job_desc = div[0].find_element(By.ID, 'jobDesc').text#.split('\n')
                                    #job_attr.append(job_desc)
                                    #job_attrs.append(job_attr)
                                
                            #else: # if post is delete
                                job_attr = [job_title] + [job_loc] + job_attr # puts title at the begining of the list
                                job_attr.append(job_desc) # Append job_description
                                job_attrs.append(job_attr) # Append job details as list inside another list 
                                
                                scraped = scraped + 1
                                
                                browser.quit()
                                smile = smile +1
                                if smile == 1:
                                    print('\nScraping away...\n')
                                    print('0  0\n')
                                    print('\__/\n')
                                if smile % 100 == 0:
                                    print("{} jobs have been scraped so far...\n".format(scraped))
                                    print('0  0\n')
                                    print('\__/\n')
                                
                                # Save progress (every job info)
                                #if smile % 500 == 0: # Every 500 jobs save as pickle
                                    # Store data (serialize)
                                with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                                    pickle.dump(job_attrs, f)
                                pp = 1

                            elif IndexError:
                                print('\nJob {} has been deleted'.format(smile))
                                not_scraped = not_scraped + 1

                                browser.quit()
                                smile = smile +1
                                if smile == 1:
                                    print('\nScraping away...\n')
                                    print('0  0\n')
                                    print('\__/\n')
                                if smile % 100 == 0:
                                    print("{} jobs have been scraped so far...\n".format(scraped))
                                    print('0  0\n')
                                    print('\__/\n')
                                
                                # Save progress (every job info)
                                #if smile % 500 == 0: # Every 500 jobs save as pickle
                                    # Store data (serialize)
                                with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                                    pickle.dump(job_attrs, f)
                                pp = 1
                            
                                
                    except Exception:
                        time.sleep(60)
                        try:
                            if del_div[6].text.startswith('We require users to verify the reCaptcha below to view deleted positions'):
                    
                                print('\nJob {} has been deleted'.format(smile))
                                not_scraped = not_scraped + 1

                                browser.quit()
                                smile = smile +1
                                if smile == 1:
                                    print('\nScraping away...\n')
                                    print('0  0\n')
                                    print('\__/\n')
                                if smile % 100 == 0:
                                    print("{} jobs have been scraped so far...\n".format(scraped))
                                    print('0  0\n')
                                    print('\__/\n')
                                
                                # Save progress (every job info)
                                #if smile % 500 == 0: # Every 500 jobs save as pickle
                                    # Store data (serialize)
                                with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                                    pickle.dump(job_attrs, f)
                                pp = 1
                        
                        except Exception:

                            browser.implicitly_wait(7)
                            browser.quit()
                            pp = 0
                            aa = aa + 1
                            print('\naa is \n', aa)
                            if aa == 5: #Let it try a few times
                                print('Pausing for 30 minutes')
                                time.sleep(1800) 
                            
                            if aa == 6:
                                print('Pausing for 3 hours!')
                                time.sleep(10800)
                            
                            if aa > 6:
                                print('Pause for 2 min intervals until it works')
                                time.sleep(120)
                            
                            if aa == 12:
                                print('Pause for 3 hours')
                                time.sleep(10800)

                    
        
        


        # else: # if exception not related to captcha, than simply info doesn't exist
        #     not_scraped = not_scraped + 1

if not_scraped > 0:

    if not_scraped == 1:
        print('Unfortunately, information for {} job was unable to be scraped.'.format(not_scraped))
    
    elif not_scraped > 1:
        print('Unfortunately, information for {} jobs was unable to be scraped.'.format(not_scraped))

print("\nInformation on a total of {} jobs has been scraped\n".format(scraped))

print("\nLenght of job_attrs is: {}\n".format(len(job_attrs)))

# Done scrpaing job attributes

##################################################


Successfuly bypassed Captcha 1!


Successfuly bypassed Captcha 2!

Looking for job link


Successfuly bypassed Captcha 1!


Successfuly bypassed Captcha 2!

Looking for job link


Successfuly bypassed Captcha 1!


Successfuly bypassed Captcha 2!

Looking for job link


Successfuly bypassed Captcha 1!


Successfuly bypassed Captcha 2!

Looking for job link


Successfuly bypassed Captcha 1!


Successfuly bypassed Captcha 2!

Looking for job link

Pausing 20 minutes, buster API overused!
Trying again

Successfuly bypassed Captcha 1!


Successfuly bypassed Captcha 2!

Looking for job link


Successfuly bypassed Captcha 1!


Successfuly bypassed Captcha 2!

Looking for job link


Successfuly bypassed Captcha 1!


Successfuly bypassed Captcha 2!

Looking for job link

Pausing 20 minutes, buster API overused!
Trying again

Successfuly bypassed Captcha 1!


Successfuly bypassed Captcha 2!

Looking for job link

Trying again

Successfuly bypassed Captcha 1!


Successfuly bypassed Captcha 2!

L

IndexError: list index out of range

Exception came up, so we checked to see if captcha was there, however not there ==> different never have had before exception
Hence ==> NoSuchElementException

In [14]:
print(scraped, not_scraped, smile)

4002 98 4100


In [15]:
len(job_links) # so missing just a few, Oh we're good

5201

rerunning on those we missed

In [16]:
for link in job_links[4100:]: #start at job 212, bc job 1 is index 0
    t = random.randint(8,10)
    time.sleep(t)
    browser = dc.initialize_driver()
    browser.get(link)

    try:
        # wait for page to load
        WebDriverWait(browser, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'row'))
        )
        del_div = browser.find_elements(By.CLASS_NAME, 'row')
        # Check if post has been deleted
        if del_div[6].text != 'We require users to verify the reCaptcha below to view deleted positions.\nRelated Searches:\nBusiness and Financial Services\nCreate your free job search account\nReceive new jobs by email\nPost your resume/CV\nTrack your applications\nJoin Now\nHave an account? Sign in':

            job_title = browser.find_element(By.ID, 'jobtitle-header').text
            job_loc = browser.find_element(By.CLASS_NAME, 'job-loc').text
            div = browser.find_elements(By.ID, 'jobAttrib')
            job_attr = div[0].find_element(By.CLASS_NAME, 'job-info').text.split('\n') # get job data
       
            div = browser.find_elements(By.ID, 'job')
            job_desc = div[0].find_element(By.ID, 'jobDesc').text#.split('\n')
            
            job_attr = [job_title] + [job_loc] + job_attr # puts title at the begining of the list
            job_attr.append(job_desc) # Append job_description
            job_attrs.append(job_attr) # Append job details as list inside another list 
            
            scraped = scraped + 1

            browser.quit()
            smile = smile +1
            if smile == 1:
                print('\nScraping away...\n')
                print('0  0\n')
                print('\__/\n')
            if smile % 100 == 0:
                print("{} jobs have been scraped so far...\n".format(scraped))
                print('0  0\n')
                print('\__/\n')
            
            # Save progress (every job info)
            #if smile % 500 == 0: # Every 500 jobs save as pickle
                # Store data (serialize)
            with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                pickle.dump(job_attrs, f)

        elif IndexError:
            print('\nJob {} has been deleted'.format(smile))
            not_scraped = not_scraped + 1

            browser.quit()
            smile = smile +1
            if smile == 1:
                print('\nScraping away...\n')
                print('0  0\n')
                print('\__/\n')
            if smile % 100 == 0:
                print("{} jobs have been scraped so far...\n".format(scraped))
                print('0  0\n')
                print('\__/\n')
            
            # Save progress (every job info)
            #if smile % 500 == 0: # Every 500 jobs save as pickle
                # Store data (serialize)
            with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                pickle.dump(job_attrs, f)

    except Exception: # If cannot find element

        # # wait for page to load
        # WebDriverWait(browser, 20).until(
        #     EC.presence_of_element_located((By.CLASS_NAME, 'iframe'))
        # ) 
               
        # See if exception is related to captcha
        if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):
            
            #bypassFullCaptcha()
            bypassCaptcha1()
            
            # Check if we get second captcha check
            if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):
                
                bypassCaptcha2()
                time.sleep(10)
        
        try:
            print('Looking for job info {}\n'.format(smile))
            print('\nNot scraped is', not_scraped)
            # Successfully moved out of Captcha

            # wait for page to load
            WebDriverWait(browser, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'row'))
            )
            del_div = browser.find_elements(By.CLASS_NAME, 'row')
            # Check if post has been deleted
            if del_div[6].text != 'We require users to verify the reCaptcha below to view deleted positions.\nRelated Searches:\nBusiness and Financial Services\nCreate your free job search account\nReceive new jobs by email\nPost your resume/CV\nTrack your applications\nJoin Now\nHave an account? Sign in':

                job_title = browser.find_element(By.ID, 'jobtitle-header').text
                job_loc = browser.find_element(By.CLASS_NAME, 'job-loc').text
                div = browser.find_elements(By.ID, 'jobAttrib')
                job_attr = div[0].find_element(By.CLASS_NAME, 'job-info').text.split('\n') # get job data
                        #job_attr = [job_title] + job_attr
                div = browser.find_elements(By.ID, 'job')
                job_desc = div[0].find_element(By.ID, 'jobDesc').text#.split('\n')
                    #job_attr.append(job_desc)
                    #job_attrs.append(job_attr)
                
            #else: # if post is delete
                job_attr = [job_title] + [job_loc] + job_attr # puts title at the begining of the list
                job_attr.append(job_desc) # Append job_description
                job_attrs.append(job_attr) # Append job details as list inside another list 
                
                scraped = scraped + 1
                
                browser.quit()
                smile = smile +1
                if smile == 1:
                    print('\nScraping away...\n')
                    print('0  0\n')
                    print('\__/\n')
                if smile % 100 == 0:
                    print("{} jobs have been scraped so far...\n".format(scraped))
                    print('0  0\n')
                    print('\__/\n')
                
                # Save progress (every job info)
                #if smile % 500 == 0: # Every 500 jobs save as pickle
                    # Store data (serialize)
                with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                    pickle.dump(job_attrs, f)

            elif IndexError:
                print('\nJob {} has been deleted'.format(smile))
                not_scraped = not_scraped + 1

                browser.quit()
                smile = smile +1
                if smile == 1:
                    print('\nScraping away...\n')
                    print('0  0\n')
                    print('\__/\n')
                if smile % 100 == 0:
                    print("{} jobs have been scraped so far...\n".format(scraped))
                    print('0  0\n')
                    print('\__/\n')
                
                # Save progress (every job info)
                #if smile % 500 == 0: # Every 500 jobs save as pickle
                    # Store data (serialize)
                with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                    pickle.dump(job_attrs, f)
            
        except Exception:

            try:
                if del_div[6].text.startswith('We require users to verify the reCaptcha below to view deleted positions'):
                    
                    print('\nJob {} has been deleted'.format(smile))
                    not_scraped = not_scraped + 1

                    browser.quit()
                    smile = smile +1
                    if smile == 1:
                        print('\nScraping away...\n')
                        print('0  0\n')
                        print('\__/\n')
                    if smile % 100 == 0:
                        print("{} jobs have been scraped so far...\n".format(scraped))
                        print('0  0\n')
                        print('\__/\n')
                    
                    # Save progress (every job info)
                    #if smile % 500 == 0: # Every 500 jobs save as pickle
                        # Store data (serialize)
                    with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                        pickle.dump(job_attrs, f)
            
            except Exception:
            # If exception is not b/c job is deleted then we need to pause
                # API limit exceeded and need to wait, pause 20 minutes and try again
                browser.quit()
                print('Pausing 20 minutes, buster API overused!')
                time.sleep(1200) # wait 20 minutes :( looooooong

                pp = 0
                aa = 0
                while pp < 1:
                    try:
                        browser = dc.initialize_driver()
                        browser.get(link)

                        #browser.switch_to.default_content()
                        
                        if browser.find_elements(By.TAG_NAME,'iframe')[0].text.startswith('Request unsuccessful.'):
                            print('Trying again on job ', smile)

                            time.sleep(10)

                            bypassCaptcha1()

                            bypassCaptcha2()
                            time.sleep(10)

                            print('Looking for job info {}\n'.format(smile))
                            print('\nNot scraped is', not_scraped)
                                
                            # Successfully moved out of Captcha

                            # wait for page to load
                            WebDriverWait(browser, 20).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'row'))
                            )
                            del_div = browser.find_elements(By.CLASS_NAME, 'row')
                            # Check if post has been deleted
                            if del_div[6].text != 'We require users to verify the reCaptcha below to view deleted positions.\nRelated Searches:\nBusiness and Financial Services\nCreate your free job search account\nReceive new jobs by email\nPost your resume/CV\nTrack your applications\nJoin Now\nHave an account? Sign in':

                                job_title = browser.find_element(By.ID, 'jobtitle-header').text
                                job_loc = browser.find_element(By.CLASS_NAME, 'job-loc').text
                                div = browser.find_elements(By.ID, 'jobAttrib')
                                job_attr = div[0].find_element(By.CLASS_NAME, 'job-info').text.split('\n') # get job data
                                        #job_attr = [job_title] + job_attr
                                div = browser.find_elements(By.ID, 'job')
                                job_desc = div[0].find_element(By.ID, 'jobDesc').text#.split('\n')
                                    #job_attr.append(job_desc)
                                    #job_attrs.append(job_attr)
                                
                            #else: # if post is delete
                                job_attr = [job_title] + [job_loc] + job_attr # puts title at the begining of the list
                                job_attr.append(job_desc) # Append job_description
                                job_attrs.append(job_attr) # Append job details as list inside another list 
                                
                                scraped = scraped + 1
                                
                                browser.quit()
                                smile = smile +1
                                if smile == 1:
                                    print('\nScraping away...\n')
                                    print('0  0\n')
                                    print('\__/\n')
                                if smile % 100 == 0:
                                    print("{} jobs have been scraped so far...\n".format(scraped))
                                    print('0  0\n')
                                    print('\__/\n')
                                
                                # Save progress (every job info)
                                #if smile % 500 == 0: # Every 500 jobs save as pickle
                                    # Store data (serialize)
                                with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                                    pickle.dump(job_attrs, f)
                                pp = 1

                            elif IndexError:
                                print('\nJob {} has been deleted'.format(smile))
                                not_scraped = not_scraped + 1

                                browser.quit()
                                smile = smile +1
                                if smile == 1:
                                    print('\nScraping away...\n')
                                    print('0  0\n')
                                    print('\__/\n')
                                if smile % 100 == 0:
                                    print("{} jobs have been scraped so far...\n".format(scraped))
                                    print('0  0\n')
                                    print('\__/\n')
                                
                                # Save progress (every job info)
                                #if smile % 500 == 0: # Every 500 jobs save as pickle
                                    # Store data (serialize)
                                with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                                    pickle.dump(job_attrs, f)
                                pp = 1
                            
                                
                    except Exception:
                        time.sleep(60)
                        try:
                            if del_div[6].text.startswith('We require users to verify the reCaptcha below to view deleted positions'):
                    
                                print('\nJob {} has been deleted'.format(smile))
                                not_scraped = not_scraped + 1

                                browser.quit()
                                smile = smile +1
                                if smile == 1:
                                    print('\nScraping away...\n')
                                    print('0  0\n')
                                    print('\__/\n')
                                if smile % 100 == 0:
                                    print("{} jobs have been scraped so far...\n".format(scraped))
                                    print('0  0\n')
                                    print('\__/\n')
                                
                                # Save progress (every job info)
                                #if smile % 500 == 0: # Every 500 jobs save as pickle
                                    # Store data (serialize)
                                with open('Job_Attrs_'+category+'.pkl', 'wb') as f:
                                    pickle.dump(job_attrs, f)
                                pp = 1
                        
                        except Exception:

                            browser.implicitly_wait(7)
                            browser.quit()
                            pp = 0
                            aa = aa + 1
                            print('\naa is \n', aa)
                            if aa == 5: #Let it try a few times
                                print('Pausing for 30 minutes')
                                time.sleep(1800) 
                            
                            if aa == 6:
                                print('Pausing for 3 hours!')
                                time.sleep(10800)
                            
                            if aa > 6:
                                print('Pause for 2 min intervals until it works')
                                time.sleep(120)
                            
                            if aa == 12:
                                print('Pause for 3 hours')
                                time.sleep(10800)

                    
        
        


        # else: # if exception not related to captcha, than simply info doesn't exist
        #     not_scraped = not_scraped + 1

if not_scraped > 0:

    if not_scraped == 1:
        print('Unfortunately, information for {} job was unable to be scraped.'.format(not_scraped))
    
    elif not_scraped > 1:
        print('Unfortunately, information for {} jobs was unable to be scraped.'.format(not_scraped))

print("\nInformation on a total of {} jobs has been scraped\n".format(scraped))

print("\nLenght of job_attrs is: {}\n".format(len(job_attrs)))

# Done scrpaing job attributes

##################################################

Looking for job info 4106


Not scraped is 98

Job 4106 has been deleted
Looking for job info 4143


Not scraped is 99

Job 4143 has been deleted
4100 jobs have been scraped so far...

0  0

\__/

Looking for job info 4212


Not scraped is 100

Job 4212 has been deleted
Looking for job info 4267


Not scraped is 101

Job 4267 has been deleted
4198 jobs have been scraped so far...

0  0

\__/

4298 jobs have been scraped so far...

0  0

\__/

Looking for job info 4429


Not scraped is 102

Job 4429 has been deleted
Looking for job info 4480


Not scraped is 103

Job 4480 has been deleted
4396 jobs have been scraped so far...

0  0

\__/

Looking for job info 4528


Not scraped is 104

Job 4528 has been deleted
Looking for job info 4565


Not scraped is 105

Job 4565 has been deleted
4494 jobs have been scraped so far...

0  0

\__/

Looking for job info 4622


Not scraped is 106

Job 4622 has been deleted
Looking for job info 4682


Not scraped is 107

Job 4682 has been deleted
Looking

In [18]:
job_attrs[0]

['Student Success Coordinator',
 'Williamsburg, KY',
 'Type: Full-Time',
 'Posted: 04/04/2023',
 'Category: Academic Advising',
 'Job Type: Full time\nJob Number: R1281\nLocation: Williamsburg, KY\nGet Set for Cumberlands!\nJoin our team of student-focused individuals working together in order to encourage intellectual and spiritual growth, leadership and service.\nCURRENT STUDENT or EMPLOYEE? Please log into Workday and use the Career Tile to find and apply to jobs from our internal career sites.\nJob Responsibilities:\nProvide academic advising for executive graduate students in multiple academic programs.\nEffectively communicate in a timely manner with graduate students.\nWork with the Registrar, International Graduate Admissions, and Department Chairs to continue the flow process of transcript evaluation, transfer credit evaluation, admissions, and academic advising.\nRespond efficiently to emails and phone calls from academic advisees.\nComplete, maintain, and update academic pla